# HVNM Flow Curve: Steady-State $\phi$-Independence

## Introduction

At steady state under constant shear rate $\dot{\gamma}$, the HVNM exhibits a **critical vitrimer signature**: both $\sigma_E \to 0$ and $\sigma_I \to 0$ because TST-mediated bond exchange allows natural states to continuously track the deformation. This means **nanoparticles do NOT affect the steady-state flow curve** — only the D-network viscosity $\eta_D = G_D / k_{d,D}$ remains. The $\phi$-independence at steady state contrasts sharply with transient protocols (startup, creep, SAOS), where NP amplification dominates. This protocol validates the stress-free natural state evolution mechanism.

> **Handbook:** See [HVNM Protocol Derivations](../../docs/source/models/hvnm/hvnm_protocols.rst) for analytical steady-state solution and [Knowledge Extraction](../../docs/source/models/hvnm/hvnm_knowledge.rst) for cross-protocol validation.

## Learning Objectives

- Understand steady-state flow: $\sigma_E = \sigma_I = 0$ as natural states track deformation
- Decompose total stress into subnetwork contributions
- Observe $\phi$-independent flow curve (nanoparticles do NOT affect steady-state)
- Analyze why only $\sigma_D = \eta_D \dot{\gamma}$ remains at steady state

## Prerequisites
- **Notebook 01** — SAOS fundamentals
- **Notebook 03** — Startup shear transients

## Estimated Runtime
- ~30 s (analytical predictions)

In [ ]:
# Google Colab setup (run only if on Colab)
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False
    print("Running locally")

In [ ]:
# Imports
from rheojax.core.jax_config import safe_import_jax, verify_float64

jax, jnp = safe_import_jax()
verify_float64()

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.models import HVNMLocal

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9
})

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
import os
import sys

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

## Theory: Steady-State Flow and φ-Independence

At steady state under constant shear rate γ̇, the HVNM model exhibits:

$$\sigma_{ss} = \sigma_P + \sigma_E + \sigma_I + \sigma_D$$

where:

1. **P-network (permanent)**: σ_P = G_P·X_I·γ (unbounded growth with strain)
2. **E-network (exchangeable)**: σ_E → 0 (natural state tracks deformation via TST)
3. **I-network (immobilized)**: σ_I → 0 (natural state also tracks deformation via TST)
4. **D-network (dangling)**: σ_D = η_D·γ̇ with η_D = G_D/k_d_D (Newtonian viscosity)

**Critical insight**: Both E-network and I-network stresses vanish at steady state because TST-mediated bond exchange allows their natural reference states to continuously evolve with the deformation. This means:

$$\sigma_{ss} = G_P \cdot X_I \cdot \gamma + \eta_D \cdot \dot{\gamma}$$

**φ-independence at steady state**: Since σ_I = 0, the immobilized network contributes ONLY to transient response (startup, creep). The steady-state flow curve depends only on G_P, η_D, and X_I (which enters via σ_P = G_P·X_I·γ, but this term diverges with strain).

For practical flow curves, the dominant contribution is σ_D = η_D·γ̇, which is φ-independent.

The apparent viscosity is:

$$\eta_{app}(\dot{\gamma}) = \frac{\sigma_{ss}}{\dot{\gamma}} = G_P \cdot X_I \frac{\gamma}{\dot{\gamma}} + \eta_D$$

For transient startup, γ/γ̇ → t, so η_app grows linearly at short times before plateauing at η_D.

In [ ]:
# Model setup
model = HVNMLocal()
model.parameters.set_value("G_P", 5000.0)    # Permanent network modulus [Pa]
model.parameters.set_value("G_E", 3000.0)    # E-network (exchangeable) [Pa]
model.parameters.set_value("G_D", 1000.0)    # D-network (dangling) [Pa]
model.parameters.set_value("phi", 0.1)       # Nanoparticle volume fraction
model.parameters.set_value("beta_I", 3.0)    # Immobilized fraction (I-network)
model.parameters.set_value("T", 350.0)       # Temperature [K]
model.parameters.set_value("k_d_D", 10.0)    # D-network dissociation rate [1/s]
model.parameters.set_value("nu_0", 1e10)     # TST attempt frequency [1/s]
model.parameters.set_value("E_a", 80e3)      # Activation energy [J/mol]
model.parameters.set_value("V_act", 1e-5)    # Activation volume [m³/mol]

eta_D = model.G_D / model.k_d_D
phi = model.phi
X_I = 1.0 + 2.5*phi + 14.1*phi**2  # Guth-Gold

print("HVNM Model Parameters:")
print(f"  G_P = {model.G_P:.0f} Pa")
print(f"  G_E = {model.G_E:.0f} Pa")
print(f"  G_D = {model.G_D:.0f} Pa")
print(f"  φ = {model.phi:.2f}")
print(f"  β_I = {model.beta_I:.1f}")
print(f"  k_d_D = {model.k_d_D:.1f} 1/s")
print(f"  η_D = G_D/k_d_D = {eta_D:.1f} Pa·s")
print(f"  X_I(φ={phi:.2f}) = {X_I:.3f}")
print(f"  T = {model.T:.0f} K")


In [ ]:
# Flow curve with component decomposition
gamma_dot = np.logspace(-2, 2, 50)

print("Computing flow curve with subnetwork decomposition...")
result = model.predict_flow_curve(gamma_dot, return_components=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Flow curve: stress vs shear rate
ax1.loglog(gamma_dot, result["stress"], "k-", lw=2, label="Total σ")
ax1.loglog(gamma_dot, result["sigma_D"], "r--", lw=1.5, label="σ_D = η_D·γ̇")
ax1.loglog(gamma_dot, np.abs(result["sigma_E"]) + 1e-10, "g:", lw=1.5, 
           label="|σ_E| (≈0 at SS)", alpha=0.7)
ax1.loglog(gamma_dot, np.abs(result["sigma_I"]) + 1e-10, "m:", lw=1.5, 
           label="|σ_I| (≈0 at SS)", alpha=0.7)
ax1.set_xlabel("γ̇ [1/s]")
ax1.set_ylabel("Stress σ [Pa]")
ax1.set_title("Flow Curve: Subnetwork Decomposition")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Apparent viscosity
eta_app = result["stress"] / gamma_dot
ax2.loglog(gamma_dot, eta_app, "k-", lw=2, label="η_app")
ax2.axhline(eta_D, color="r", ls=":", lw=1.5, 
            label=f"η_D = G_D/k_d_D = {eta_D:.1f} Pa·s")
ax2.set_xlabel("γ̇ [1/s]")
ax2.set_ylabel("η [Pa·s]")
ax2.set_title("Apparent Viscosity")
ax2.legend()
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

# Find index closest to gamma_dot = 1
idx = np.argmin(np.abs(gamma_dot - 1.0))
print(f"\nStress components at γ̇ = {gamma_dot[idx]:.2f} s⁻¹:")
print(f"  σ_total = {result['stress'][idx]:.2f} Pa")
print(f"  σ_D = {result['sigma_D'][idx]:.2f} Pa")
print(f"  σ_E = {result['sigma_E'][idx]:.2e} Pa (≈0)")
print(f"  σ_I = {result['sigma_I'][idx]:.2e} Pa (≈0)")
print(f"\n→ σ_E and σ_I vanish at steady state (natural states track deformation)")

In [ ]:
# φ-independence demonstration
phi_values = [0.0, 0.1, 0.2]
colors = ['blue', 'green', 'red']

fig, ax = plt.subplots(figsize=(8, 5))

print("\nComparing flow curves across different φ values...")
for phi_val, color in zip(phi_values, colors):
    model_temp = HVNMLocal()
    model_temp.parameters.set_value("G_P", 5000.0)
    model_temp.parameters.set_value("G_E", 3000.0)
    model_temp.parameters.set_value("G_D", 1000.0)
    model_temp.parameters.set_value("phi", phi_val)
    model_temp.parameters.set_value("beta_I", 3.0)
    model_temp.parameters.set_value("T", 350.0)
    model_temp.parameters.set_value("k_d_D", 10.0)
    model_temp.parameters.set_value("nu_0", 1e10)
    model_temp.parameters.set_value("E_a", 80e3)
    model_temp.parameters.set_value("V_act", 1e-5)
    
    r = model_temp.predict_flow_curve(gamma_dot, return_components=True)
    X = 1.0 + 2.5*phi_val + 14.1*phi_val**2
    
    ax.loglog(gamma_dot, r["stress"], "-", lw=1.5, color=color, 
              label=f"φ = {phi_val:.1f} (X_I = {X:.3f})")
    print(f"  φ = {phi_val:.1f}: σ_total(γ̇=1) = {r['stress'][idx]:.2f} Pa")

ax.loglog(gamma_dot, eta_D * gamma_dot, color="black", ls=":", lw=1, alpha=0.5,
          label="η_D·γ̇ (theoretical)")
ax.set_xlabel("γ̇ [1/s]")
ax.set_ylabel("Stress σ [Pa]")
ax.set_title("Flow Curve: φ-Independence at Steady State")
ax.legend()
ax.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

print("\n→ Flow curves overlap (φ-independent) because σ_E = σ_I = 0 at steady state")

## Key Takeaways

1. **E-network and I-network vanish**: $\sigma_E = \sigma_I \to 0$ at steady state because TST allows natural states to track deformation
2. **D-network dominance**: Only $\sigma_D = \eta_D \dot{\gamma}$ remains (Newtonian viscosity)
3. **$\phi$-independent flow curve**: Nanoparticles do NOT affect steady-state because $\sigma_I = 0$
4. **Transient vs steady-state**: NP effects appear in startup/creep (transient) but vanish at SS
5. **Apparent viscosity**: Plateaus at $\eta_D = G_D / k_{d,D}$ at high shear rates

## Further Reading

**Handbook Documentation:**
- [HVNM Protocol Derivations](../../docs/source/models/hvnm/hvnm_protocols.rst) — Analytical steady-state solution
- [HVNM Knowledge Extraction](../../docs/source/models/hvnm/hvnm_knowledge.rst) — Cross-protocol validation, why $\sigma_I = 0$

**Key References:**
1. Karim, M.R., Vernerey, F. & Sain, T. (2025). *Macromolecules*, 58(10), 4899-4912.

## Next Notebooks

- **Notebook 06**: LAOS — Payne effect at $\gamma_c / X_I$
- **Notebook 07**: Limiting cases — Factory methods and $\phi=0$ HVM recovery
- **Notebook 08**: Data intake and QC workflows